In [75]:
#a hand gesture recognition model that can accurately identify and classify different hand gestures from image or video data, enabling intuitive human-computer interaction and gesture-based control systems.
import cv2
import mediapipe as mp
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import pickle 
import os 
import random
from cvzone.HandTrackingModule import HandDetector
pd.set_option("display.max_columns",None)
plt.style.use('ggplot')


In [76]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import math
import time 
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
imgSize =500
offset = 25
counter = 0
folder = 'data/A'
while True:
    success, img = cap.read()
    hands, img = detector.findHands(img)
    #try and except to escape situation where hand gets out of frame nd casuse imshow to crash 
    try:
        if hands:
            hand = hands[0]
            print(hand)
            x,y,w,h = hand['bbox']
            imageWhit = np.ones((imgSize,imgSize,3),np.uint8)*255 # multiply ny 255 to make the array of ones white
            imgCrop = img[y-offset:y+offset+h,x-offset:x+offset+w] # 
            
            
            #pin imgcrop to center og imgwhite
            aspectratio = h/w
            if aspectratio >1:
                k = imgSize/h
                wcal= math.ceil(k *w)
                
                imgResize= cv2.resize(imgCrop,(wcal, imgSize))
                imgResizeShape = imgResize.shape
                wGap = math.ceil((imgSize - wcal)/2)
                imageWhit[:imgResizeShape[0],wGap:wcal+wGap]=imgResize# pin image resize to the specified cordinates of imgwhite
            else:
                k = imgSize/w
                hcal= math.ceil(k *h)
                imgResize= cv2.resize(imgCrop,( imgSize, hcal))
                imgResizeShape = imgResize.shape
                hGap = math.ceil((imgSize - hcal)/2)
                imageWhit[hGap:hcal+hGap,:imgResizeShape[0]]=imgResize
            
            cv2.imshow('crop',imgCrop)
            cv2.imshow("imgwht",imageWhit)
    except Exception:
        pass
    cv2.imshow("image",img)
    key = cv2.waitKey(33)
    if key & 0xFF == ord('q'):
        break
    elif key == ord('s'):
        counter+=1
        cv2.imwrite(f"{folder}/image_{time.time()}.jpg", imageWhit)
        print(counter)
cap.release()
cv2.destroyAllWindows()
    
   
    

C:\Users\Benwari Ezekiel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


{'lmList': [[192, 414, 0], [245, 425, -30], [312, 408, -43], [365, 395, -55], [408, 386, -66], [327, 293, -23], [361, 238, -45], [386, 204, -64], [407, 176, -79], [298, 266, -25], [332, 196, -48], [359, 152, -69], [384, 117, -83], [262, 257, -32], [292, 188, -56], [318, 144, -75], [343, 107, -86], [219, 264, -41], [233, 209, -67], [244, 172, -79], [256, 138, -85]], 'bbox': (192, 107, 216, 318), 'center': (300, 266), 'type': 'Right'}
{'lmList': [[180, 391, 0], [243, 402, -23], [311, 391, -38], [363, 381, -53], [405, 381, -69], [326, 273, -18], [370, 222, -38], [396, 191, -56], [420, 164, -70], [299, 247, -24], [343, 181, -42], [374, 137, -60], [400, 103, -73], [266, 236, -34], [303, 170, -52], [328, 126, -69], [352, 91, -80], [224, 239, -46], [242, 185, -65], [255, 150, -74], [268, 119, -79]], 'bbox': (180, 91, 240, 311), 'center': (300, 246), 'type': 'Right'}
{'lmList': [[178, 385, 0], [240, 395, -24], [307, 386, -40], [361, 378, -56], [406, 376, -72], [329, 270, -16], [373, 220, -37],

In [77]:
import cv2 
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import pandas as pd
import os
import mediapipe as mp

# Initialize Mediapipe Hand Detector and Drawing Utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawings = mp.solutions.drawing_utils

# Base Directory and Categories for Image Files
base_url = r'C:\Users\Benwari Ezekiel\Documents\code\archive (1)\leapGestRecog\00'
categories = ['01_palm', '02_l', '03_fist', '04_fist_moved', '05_thumb', '06_index', '07_ok', '08_palm_moved', '09_c', '10_down']
data = []
labels = []
# Loop through Categories and Images
for category in categories:
    path = os.path.join(base_url, category)
    for img in os.listdir(path):
        image_page = os.path.join(path, img)
        img_file = cv2.imread(image_page, 0)  # Read image in grayscale

        # Correct Color Conversion
        img_rgb = cv2.cvtColor(img_file, cv2.COLOR_GRAY2RGB)  # Convert Grayscale to RGB
        
        try: 
            img_result = hands.process(img_rgb)  # Process the RGB image for hand detection
            
            # Check if any hands are detected
            if img_result.multi_hand_landmarks:
                for hand_landmark in img_result.multi_hand_landmarks:
                    # Correct usage of draw_landmarks with the original image
                    mp_drawings.draw_landmarks(img_rgb, hand_landmark, mp_hands.HAND_CONNECTIONS)
                    
                    # Extract landmarks and flatten them
                    landmarks = []
                    for lm in hand_landmark.landmark:
                        landmarks.extend([lm.x, lm.y, lm.z])
                    
                    hand_data = pd.DataFrame(np.array(landmarks).flatten()).T  # Create DataFrame for the current hand data
                    data.append(hand_data)  # Append hand data to the list
                    labels.append(category)
            
            # Display the image with drawn landmarks
            cv2.imshow('image', img_rgb)  
            key = cv2.waitKey(33)
            if key == ord('q'):  # Quit on pressing 'q'
                break 
        
        except Exception as e:
            print("There is a problem:", e)  # Print exception message for better debugging

    cv2.destroyAllWindows()


C:\Users\Benwari Ezekiel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Benwari Ezekiel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Benwari Ezekiel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\googl

In [78]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import pickle 
import array
le = LabelEncoder()
data2 = np.squeeze(data, axis=1)
df = pd.DataFrame(data2)
#print(df)
labels = pd.DataFrame(labels)
print(labels)
labels = le.fit_transform(labels)
x= df
y = labels.reshape(-1,1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
print(x.shape)
print(y.shape)
model.fit(x_train, y_train)
y_pred= model.predict(x_test)
with open('randomfroeestclssifier_hand_guesture', 'wb') as f:
    pickle.dump(model,f)


print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

          0
0   01_palm
1   01_palm
2   01_palm
3   01_palm
4   01_palm
5   01_palm
6   01_palm
7   01_palm
8   01_palm
9   01_palm
10  01_palm
11  01_palm
12  01_palm
13  01_palm
14  01_palm
15  01_palm
16  01_palm
17  01_palm
18  01_palm
19  01_palm
20  01_palm
21  01_palm
22  01_palm
23  01_palm
24  01_palm
25  01_palm
26  01_palm
27  01_palm
28  01_palm
29  01_palm
30  01_palm
31  01_palm
32  01_palm
33  01_palm
34  01_palm
35  01_palm
36  01_palm
37  01_palm
38     02_l
39     02_l
40     02_l
41    07_ok
42    07_ok
(43, 63)
(43, 1)


C:\Users\Benwari Ezekiel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Benwari Ezekiel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         1

    accuracy                           1.00         9
   macro avg       1.00      1.00      1.00         9
weighted avg       1.00      1.00      1.00         9

Accuracy Score: 1.0
